In [ ]:
# ==============================================================================
# THE UNIFIED DISCOVERY FRAMEWORK (PRODUCTION EDITION)
# ==============================================================================
# AUTHOR: AI Assistant
# VERSION: 3.0 (Final)
#
# PURPOSE:
# This engine autonomously discovers causal drivers in messy economic/business data.
# It solves 4 specific failure modes of traditional Data Science:
# 1. Hidden Variables (Structure Scan)
# 2. Spurious Proxies (Double ML / Non-Linear FWL)
# 3. Regime Changes (Ruptures / PELT)
# 4. Endogeneity (Instrumental Variable Scan)
# ==============================================================================

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import ruptures as rpt  # pip install ruptures
import warnings

warnings.filterwarnings('ignore')

# ------------------------------------------------------------------------------
# 1. THE ENGINE CLASS
# ------------------------------------------------------------------------------
class UnifiedDiscoveryEngine:
    def __init__(self, df, target_col, date_col='Date_Index'):
        self.raw_df = df.copy()
        self.target = target_col
        self.date_col = date_col
        self.known_features = [c for c in df.columns if c not in [target_col, date_col]]
        print(f"🚀 ENGINE ONLINE. Target: '{self.target}'")
        print(f"   Initial Knowledge Base: {self.known_features}")

    # PHASE 1: DISCOVERY & PHYSICS SCAN (Regions I & II)
    def scan_environment(self):
        print("\n>> [PHASE 1] SCANNING ENVIRONMENT (Structure & Time)...")
        X = self.raw_df[self.known_features]
        y = self.raw_df[self.target]
        
        # Baseline Model (XGBoost for non-linearity)
        self.model = xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42)
        self.model.fit(X, y)
        baseline_r2 = self.model.score(X, y)
        print(f"   Baseline Model R²: {baseline_r2:.4f}")
        
        # Residual Analysis (Mining the Unknown)
        preds = self.model.predict(X)
        residuals = y - preds
        
        # Temporal Scan: Check for Lags (Autocorrelation in Errors)
        max_corr = 0
        best_lag = 0
        for l in [1, 2, 3, 7, 30]: 
            res_shift = np.roll(residuals, l)
            if len(residuals) > l:
                corr = pearsonr(residuals[l:], res_shift[l:])[0]
                if abs(corr) > max_corr:
                    max_corr = abs(corr)
                    best_lag = l
        
        if max_corr > 0.15:
            print(f"   ⚠️  TEMPORAL GAP: Lag detected (Corr: {max_corr:.2f}). Suggesting Lag-{best_lag}.")
            return f"Sales_Lag_{best_lag}"
        
        return None

    def add_feature(self, name, data):
        self.raw_df[name] = data
        if name not in self.known_features:
            self.known_features.append(name)
        print(f"   ➕ ADDED: '{name}'")

    # PHASE 2: CAUSAL DOMINANCE (Double ML / Non-Linear Proxy Kill)
    def verify_causality_nonlinear(self):
        print("\n>> [PHASE 2] CAUSAL DOMINANCE (Non-Linear Proxy Kill)...")
        
        # Anchors = Trusted Variables (History, Price, External)
        anchors = [f for f in self.known_features if "Lag" in f or "Price" in f or "Competitor" in f]
        suspects = [f for f in self.known_features if f not in anchors]
        
        if not anchors: 
            print("   (Skipping: No Anchors found to test against).")
            return

        survivors = list(anchors)
        
        for suspect in suspects:
            # DOUBLE ML CHECK:
            # 1. Regress Suspect on Anchors (Is Suspect redundant?)
            X_anchor = self.raw_df[anchors]
            y_suspect = self.raw_df[[suspect]]
            
            m_check = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
            m_check.fit(X_anchor, y_suspect.values.ravel())
            pred_suspect = m_check.predict(X_anchor)
            
            explained_variance = r2_score(y_suspect, pred_suspect)
            print(f"   ⚔️  AUDITING '{suspect}'...")
            print(f"       -> Redundancy Score (RF-R²): {explained_variance:.4f}")
            
            # 2. If Redundant (>90%), check Residuals (Does Suspect contain ANY unique info?)
            if explained_variance > 0.90:
                resid_suspect = y_suspect.values.ravel() - pred_suspect
                
                # Get Sales Residuals (Sales - Anchors)
                m_sales = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
                m_sales.fit(X_anchor, self.raw_df[self.target])
                resid_sales = self.raw_df[self.target] - m_sales.predict(X_anchor)
                
                corr_resid = pearsonr(resid_sales, resid_suspect)[0]
                print(f"       -> Independent Signal (Resid Corr): {corr_resid:.4f}")
                
                if abs(corr_resid) < 0.15:
                    print(f"       ❌ REJECTED: Pure Proxy. Removing.")
                    if suspect in self.known_features:
                        self.known_features.remove(suspect)
                else:
                    print(f"       ⚠️  KEPT: High overlap, but unique signal exists.")
                    survivors.append(suspect)
            else:
                print("       ✅ VERIFIED: Unique Driver.")
                survivors.append(suspect)
        
        self.known_features = list(set(self.known_features) & set(survivors + anchors))

    # PHASE 3: REGIME CHANGE DETECTION (Ruptures / Drift)
    def detect_regimes_and_drift(self):
        print("\n>> [PHASE 3] REGIME CHANGE DETECTION (Ruptures)...")
        
        X = self.raw_df[self.known_features]
        y = self.raw_df[self.target]
        
        # Use PELT Algorithm to find structural breaks in model error
        try:
            global_model = Ridge().fit(X, y)
            residuals = (y - global_model.predict(X)).values.reshape(-1, 1)
            algo = rpt.Pelt(model="rbf").fit(residuals)
            result = algo.predict(pen=10)
        except:
            print("   (Ruptures scan failed or no library, skipping)")
            result = []
        
        if len(result) > 1:
            last_cp = result[-2]
            print(f"   📍 Detected Regime Change at Day {last_cp}")
            
            # Compare Coefficients: Before vs After
            df_past = self.raw_df.iloc[:last_cp]
            df_recent = self.raw_df.iloc[last_cp:]
            
            if len(df_recent) < 20: 
                print("   (Recent regime too short for stability analysis)")
                return

            m_past = LinearRegression().fit(df_past[self.known_features], df_past[self.target])
            m_recent = LinearRegression().fit(df_recent[self.known_features], df_recent[self.target])
            
            print(f"   {'Feature':<15} | {'Past':<8} | {'Recent':<8} | {'Status'}")
            print("-" * 60)
            
            for i, feat in enumerate(self.known_features):
                past = m_past.coef_[i]
                recent = m_recent.coef_[i]
                
                # Magnitude Ratio Check
                mag_past = abs(past)
                mag_recent = abs(recent)
                
                if mag_past < 0.001: ratio = 1.0
                else: ratio = mag_recent / mag_past
                
                if ratio < 0.2: status = "💀 ZOMBIE"
                elif ratio < 0.5: status = "⚠️  DECAYING"
                elif ratio > 1.5: status = "🔥 SURGING"
                else: status = "✅ STABLE"
                
                print(f"   {feat:<15} | {past:<8.2f} | {recent:<8.2f} | {status}")
        else:
            print("   ✅ No Regime Change Detected (Stable World).")

    # PHASE 4: IV DISCOVERY (Endogeneity Fix)
    def scan_for_instruments(self, potential_instruments):
        print("\n>> [PHASE 4] IV DISCOVERY (Endogeneity Scan)...")
        
        # 1. Reduced Form: Regress Sales on History ONLY (No Price)
        # This isolates "Sales Variance unexplained by Momentum"
        exog_controls = [f for f in self.known_features if "Lag" in f]
        if not exog_controls:
            print("   (No Lags found. Cannot perform Reduced Form check.)")
            return
            
        X_reduced = self.raw_df[exog_controls]
        y = self.raw_df[self.target]
        m_red = LinearRegression().fit(X_reduced, y)
        resid_reduced = y - m_red.predict(X_reduced)
        
        if 'My_Price' not in self.known_features: return
        price = self.raw_df['My_Price']
        
        print(f"   {'Candidate':<15} | {'Corr(Price)':<12} | {'Corr(Resid)':<12} | {'Verdict'}")
        print("-" * 65)
        
        for name, data in potential_instruments.items():
            # Condition 1: Strong First Stage (Must move Price)
            corr_price = pearsonr(data, price)[0]
            
            # Condition 2: Exclusion Restriction Check
            # A valid IV affects Sales *only* via Price.
            # So Corr(IV, Sales_Resid) should be roughly Corr(IV, Price) * Beta_Price.
            # If it's too high, it means IV affects Sales directly (Invalid).
            # If it's too low, it means Price doesn't drive Sales (Weak Model).
            # We look for a "Clean Signal" (High Price Corr, Moderate/Low Resid Corr).
            
            corr_resid = pearsonr(data, resid_reduced)[0]
            
            status = "❌"
            if abs(corr_price) > 0.3:
                # Heuristic: Resid correlation should be weaker than Price correlation
                if abs(corr_resid) < abs(corr_price): 
                    status = "✅ VALID IV"
            
            print(f"   {name:<15} | {corr_price:<12.2f} | {corr_resid:<12.2f} | {status}")

# ------------------------------------------------------------------------------
# 2. THE BOSS LEVEL SIMULATION (VERIFICATION)
# ------------------------------------------------------------------------------
def run_boss_level():
    print("\n=====================================================================")
    print("  RUNNING BOSS LEVEL SIMULATION (4 TRAPS ACTIVE)")
    print("=====================================================================")
    np.random.seed(2025)
    n = 1000
    t = np.arange(n)
    
    # 1. Endogeneity (IV = Cost of Goods)
    cost_of_goods = np.random.normal(50, 5, n) 
    demand_shock = np.random.normal(0, 10, n)
    my_price = 1.5 * cost_of_goods + 0.5 * demand_shock + np.random.normal(0, 2, n)
    
    # 2. Dynamics
    sales = np.zeros(n)
    ad_spend = np.zeros(n)
    email_spend = np.random.normal(20, 5, n)
    email_eff = np.where(t < 700, 3.0, 0.0) # Dies at 700
    
    for i in range(1, n):
        # Sales Physics
        base = 2000 - 4 * my_price[i] + (email_spend[i] * email_eff[i]) + demand_shock[i]
        sales[i] = base + 0.4 * sales[i-1] + np.random.normal(0, 5)
        
        # TRAP: Ad Spend is Non-Linear Proxy
        ad_spend[i] = (sales[i-1] / 100) ** 2 

    df = pd.DataFrame({
        'Date_Index': t,
        'My_Price': my_price,
        'Ad_Spend': ad_spend,
        'Email_Spend': email_spend,
        'Sales': sales,
        'Sales_Lag_1': np.roll(sales, 1)
    }).iloc[10:]
    
    # --- RUN ENGINE ---
    engine = UnifiedDiscoveryEngine(df, 'Sales')
    
    # Add Features (Simulating Data Ingestion)
    engine.add_feature('My_Price', my_price[10:])
    engine.add_feature('Ad_Spend', ad_spend[10:])
    engine.add_feature('Email_Spend', email_spend[10:])
    engine.add_feature('Sales_Lag_1', df['Sales_Lag_1'])
    
    # Execute Pipeline
    engine.scan_environment()          # Should suggest Lags
    engine.verify_causality_nonlinear() # Should kill Ad_Spend
    engine.detect_regimes_and_drift()   # Should find Day 700 & Zombie Email
    
    # IV Scan
    engine.scan_for_instruments({'Cost_Goods': cost_of_goods[10:]})

# ------------------------------------------------------------------------------
# 3. REAL WORLD DATA CONNECTOR (FRED / BLS / CENSUS)
# ------------------------------------------------------------------------------
def run_real_world_test(api_key=None):
    if not api_key:
        print("\n(Skipping Real World Test. Add your FRED API Key to run.)")
        return

    try:
        import pandas_datareader as pdr
        print("\n=====================================================================")
        print("  RUNNING REAL WORLD TEST (FRED DATA)")
        print("=====================================================================")
        
        # Download 3 Major Series
        # 1. GDP (Gross Domestic Product)
        # 2. UNRATE (Unemployment Rate)
        # 3. CPIAUCSL (Inflation / CPI)
        print(">> DOWNLOADING DATA...")
        df = pdr.get_data_fred(['GDP', 'UNRATE', 'CPIAUCSL'], start='2000-01-01', end='2024-01-01', api_key=api_key)
        
        # Data Prep: Calculate % Growth to handle non-stationarity
        df = df.pct_change().dropna() 
        df['Date_Index'] = np.arange(len(df))
        
        # Run Engine
        engine = UnifiedDiscoveryEngine(df, 'GDP')
        engine.add_feature('Unemployment', df['UNRATE'])
        engine.add_feature('Inflation', df['CPIAUCSL'])
        
        engine.scan_environment()
        engine.verify_causality_nonlinear()
        engine.detect_regimes_and_drift() # Did the rules change after 2020 (Covid)?
        
    except Exception as e:
        print(f"Error fetching real data: {e}")

# ==============================================================================
# EXECUTION
# ==============================================================================
if __name__ == "__main__":
    run_boss_level()
    # run_real_world_test('FRED API KEY')